In [1]:
from tkinter import filedialog
import cv2
from Transformaciones import Transformaciones
import numpy as np

ruta = filedialog.askopenfilename(title="Selecciona una imagen")
img_original = cv2.imread(ruta)

if img_original is None:
    print("Error: no se pudo cargar la imagen")
    exit()
    
def nothing(x):
    pass

img_original = cv2.resize(img_original, None, fx=0.2, fy=0.2)

roi_cords = None
roi_img = None

def select_roi(img):
    global roi_cords, roi_img
    
    print("Selecionar el Area de Interes (Pulse ENTER para continuar)")
    roi_temp = cv2.selectROI("Selecionar ROI", img, False, False)
    cv2.destroyWindow("Selecionar ROI")
    
    if roi_temp[2] > 0 and roi_temp[3] > 0:
        roi_cords = roi_temp
        x, y, w, h = roi_cords
        roi_img = img[y:y+h, x:x+w]
        print(f"ROI: = x={x}, y={y}, w={w}, h={h}")
        return True
    return False

select_roi(img_original)

img_transformada = None

transformaciones = Transformaciones(img_original)
transformaciones.distorcion()
img_transformada = transformaciones.distorcioned
#transformaciones.transformacion()
#img_transformada = transformaciones.transformed

params_SIFT = {
    "nfeatures": 500,
    "nOctaveLayers": 3,
    "contrastThreshold": 0.04,
    "edgeThreshold": 10,    
    "sigma": 1.6
}

def configurate_SIFT(img):
    cv2.namedWindow('SIFT configuration')
    
    cv2.createTrackbar('nfeatures', 'SIFT configuration', 500, 2000, nothing)
    cv2.createTrackbar('nOctaveLayers', 'SIFT configuration', 3, 5, nothing)
    cv2.createTrackbar('contrastThreshold', 'SIFT configuration', 40, 100, nothing)
    cv2.createTrackbar('edgeThreshold', 'SIFT configuration', 10, 20, nothing)
    cv2.createTrackbar('sigma', 'SIFT configuration', 16, 30, nothing)
    
    print("Ajustar los parametros del SIFT. Presionar q para continuar")
    
    while True:
        params_SIFT['nfeatures'] = nfeat = cv2.getTrackbarPos('nfeatures', 'SIFT configuration')
        params_SIFT['nOctaveLayers'] = nOc = cv2.getTrackbarPos('nOctaveLayers', 'SIFT configuration')
        params_SIFT['contrastThreshold'] = conTh = cv2.getTrackbarPos('contrastThreshold', 'SIFT configuration') / 1000.0
        params_SIFT['edgeThreshold'] = edge = cv2.getTrackbarPos('edgeThreshold', 'SIFT configuration')
        params_SIFT['sigma'] = sigma = cv2.getTrackbarPos('sigma', 'SIFT configuration') / 10.0
        
        sift = cv2.SIFT_create(nfeat, nOc, conTh, edge, sigma)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = sift.detectAndCompute(gray, None)
        
        img_with_keypoints = cv2.drawKeypoints(img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        cv2.imshow('SIFT configuration', img_with_keypoints)
        
        key = cv2.waitKey(100) & 0xFF
        if key == ord('q'):
            break
    cv2.destroyWindow('SIFT configuration')
    print("Parametros de SIFT configurados")
        
configurate_SIFT(img_original)

def detect_matches(img_trf, img_roi, num_matches = 10):
    
    if  img_trf is None or img_roi is None:
        print("Error: Falta de ROI o imagen transformada")
        return None
    
    sift = cv2.SIFT_create(
        nfeatures=params_SIFT['nfeatures'],
        nOctaveLayers=params_SIFT['nOctaveLayers'],
        contrastThreshold=params_SIFT['contrastThreshold'],
        edgeThreshold=params_SIFT['edgeThreshold'],
        sigma=params_SIFT['sigma']
    )
    
    roi_gray = cv2.cvtColor(img_roi, cv2.COLOR_BGR2GRAY)
    trf_gray = cv2.cvtColor(img_trf, cv2.COLOR_BGR2GRAY)
    
    kp_roi, des_roi = sift.detectAndCompute(roi_gray, None)
    kp_trf, des_trf = sift.detectAndCompute(trf_gray, None)
    
    print(f"Roi Keypoints: {len(kp_roi)}, Transformada Keypoints: {len(kp_trf)}")
    
    if des_roi is None or des_trf is None or len(kp_roi) < 2 or len(kp_trf) < 2:
        print("Pocas caracteristicas encontradas")
        return None
    
    flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=50))
    
    matches = flann.knnMatch(des_roi, des_trf, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
        
    print(f"Matches buenos: {len(good_matches)}")
    
    if len(good_matches) < num_matches:
        print(f"Pocos matches ({len(good_matches)} < {num_matches})")
        return None
    
    #Homografia
    src_pts = np.float32([kp_roi[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_trf[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    if M is None:
        print("No se pudo calcular la homografía")
        return None
    
    #Cuadro rojo
    h, w = roi_gray.shape
    pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)
    
    img_result = img_trf.copy()
    img_result = cv2.polylines(img_result, [np.int32(dst)], True, (0, 0, 255), 3)
    
    #Matches
    img_matches = cv2.drawMatches(img_roi, kp_roi, img_trf, kp_trf, good_matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    print("Roi Encontrado")
    return img_result, img_matches, M, len(good_matches)

resultado = detect_matches(img_transformada, roi_img, 5)

if resultado is None:
    print("No se encontraron matches suficientes")
else:
    imgen_result, imgen_matches, M, tam = resultado
    while True:
        cv2.imshow("Imagen Original", img_original)
        cv2.imshow("Imagen Matches", imgen_matches)
        cv2.imshow("Imagen Resultados", imgen_result)

        key = cv2.waitKey(1) & 0xFF  # Revisar tecla presionada
        if key == 27:  # 27 = ESC
            break
        
    cv2.destroyAllWindows()


Selecionar el Area de Interes (Pulse ENTER para continuar)
ROI: = x=182, y=332, w=83, h=72
Ajustar los parametros del SIFT. Presionar q para continuar
Parametros de SIFT configurados
Roi Keypoints: 67, Transformada Keypoints: 501
Matches buenos: 10
Roi Encontrado
